In [ ]:
from anndata import read_zarr
from os.path import join
import os

In [ ]:
import scanpy as sc
from muon import prot as pt

In [ ]:
!pwd

In [ ]:
PATH_TO_EXPORT_DIRECTORY = join("..", "..", "hubmap-publication-page", "data", "codex")
VIGNETTE_DIR = join("..", "..", "hubmap-publication-page", "vignettes", "vignette_04")

In [ ]:
adata = read_zarr(join("..", "data", "raw", "reg001_expr-anndata.zarr"))
adata

In [ ]:
# TODO: try computing new dimensionality reductions

In [ ]:
pt.pp.clr(adata)

In [ ]:
adata.X

In [ ]:
sc.pp.pca(adata)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['Cell K-Means [Mean] Expression'])

In [ ]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    MultiImageWrapper,
    OmeTiffWrapper,
    hconcat,
    BASE_URL_PLACEHOLDER
)
import json
from os.path import join
import os

In [ ]:
BASE_DIR = join("..", "data", "raw")

In [ ]:
vc = VitessceConfig(schema_version="1.0.15", name='CODEX', description='')

In [ ]:
dataset = vc.add_dataset(name='CODEX').add_object(AnnDataWrapper(
    adata_path=join(BASE_DIR, "reg001_expr-anndata.zarr"),
    obs_embedding_paths=["obsm/tsne"],
    obs_embedding_names=["t-SNE"],
    obs_locations_path="obsm/xy",
    obs_set_paths=["obs/Cell K-Means [Mean] Expression"],
    obs_set_names=["Cell K-Means [Mean] Expression"],
    obs_feature_matrix_path="X",
    coordination_values={
        "obsType": "cell",
        "featureType": "antigen",
        "featureValueType": "expression"
    }
)).add_object(MultiImageWrapper([
    OmeTiffWrapper(
        img_path=join(BASE_DIR, "ome_tiff", "mask", "reg001_mask.ome.tif"),
        offsets_path=join(BASE_DIR, "ome_tiff_offsets", "mask", "reg001_mask.offsets.json"),
        name="Segmentations",
        is_bitmask=True
    ),
    OmeTiffWrapper(
        img_path=join(BASE_DIR, "ome_tiff", "expr", "reg001_expr.ome.tif"),
        offsets_path=join(BASE_DIR, "ome_tiff_offsets", "expr", "reg001_mask.offsets.json"),
        name="Image"
    )
]))

In [ ]:
layer_controller = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset)
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
scatterplot = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="t-SNE")
marker_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
description = vc.add_view(vt.DESCRIPTION, dataset=dataset)
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset).set_props(transpose=True)
cell_set_manager = vc.add_view(vt.OBS_SETS, dataset=dataset)


In [ ]:
vc.layout(
    hconcat(layer_controller, spatial, scatterplot, marker_list)
    / hconcat(description, heatmap, cell_set_manager)
);

In [ ]:
all_views = [
    layer_controller, spatial, scatterplot, marker_list,
    description, heatmap, cell_set_manager,
]
vc.link_views(
    all_views,
    [ct.FEATURE_TYPE],
    ["antigen"]
)

In [ ]:
#vw = vc.widget()
#vw

In [ ]:
vc.web_app()

In [ ]:
os.makedirs(VIGNETTE_DIR, exist_ok=True)

In [ ]:
os.makedirs(PATH_TO_EXPORT_DIRECTORY, exist_ok=True)
config_dict = vc.export(to="files", base_url=f"{BASE_URL_PLACEHOLDER}/codex", out_dir=PATH_TO_EXPORT_DIRECTORY)

# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(join(VIGNETTE_DIR, "codex.json"), "w") as f:
    json.dump(config_dict, f)

In [ ]:
vignette_md = """---
name: Use Case 4
figures:
    - name: "Visualization"
      file: codex.json
---


## Multiplexed cyclic fluorescence images of oligo-barcoded antibodies in single cells from the human spleen

CO-Detection by indEXing (CODEX) is a spatial assay which uses oligonucleotide probes conjugated to antibodies specific to a set of antigens of interest. Through multiple rounds of hybridization, up to 50 different antigens may be imaged. We present an example of using Vitessce to explore a human spleen tissue sample in which 29 antigens were targeted. The dataset contains the 29-channel microscopy images, cell segmentations, cell-by-antigen quantification, and unsupervised clustering results. To interact with this dataset, we configure Vitessce with a spatial plot, heatmap, and controllers to select image channels, antigens, and cell clusters of interest. With multiple representations of the same data, we can choose to begin exploration by focusing on one of several entity types.

Because this dataset lacks cell type annotations, we are interested in the protein markers which define each cluster. We approach the visual analysis by selecting a cluster in the cell set controller, then searching for cluster-specific patterns in the spatial view. With clustering assignments encoded as cell segmentation mask colors, it appears that cells assigned to cluster 6 within the “Cell K-Means Mean-All-SubRegions Expression” localize to a small set of compact regions in the spatial view, suggestive of cellular neighborhoods in the spleen. When we identify cluster 6 in the heatmap view, it is clear that the marker with the highest relative expression in this cluster corresponds to the B cell marker CD20. We can confirm that CD20 is enriched in this spatial region by hiding the cell segmentation masks in the spatial plot, uncovering the underlying image data. Using the spatial layer controller, we can select the CD20 channel and verify that the fluorescence signal indeed appears in the expected regions. The heatmap and image views can help us to find other markers correlated with the CD20 spatial expression pattern.

"""

with open(join(VIGNETTE_DIR, "description.md"), "w") as f:
    f.write(vignette_md)